<a href="https://colab.research.google.com/github/hadwin-357/ProteinMPNN_breakdown/blob/main/model_utils_function_7_ProteinFeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#test model utils function
#ProteinFeature
_dist : use ca cooridinates to calculate k-nearest CA- CA neighbor distance and idx\
_rbf : use ca k-nearest neighbor distance to encode through rbf kernel\
_get_rbf: similar to _rbf, but used for other atoms such as N C O CA
## _dist function
Arg: X [B, N, 1, 3] only CA dimension ; mask [B, N]\
Functiona explain:  1. make inter-residue mask-mask_2D
2. calculate inter-residue distance   
_2.1 calculate xi-xj, yi-yj, zi-zj \
_2.2 square the value \
_2.3 sum over x, y, z  note: in orignal code,  N CA C O dimension are extract individually, then do calcualtion
3. extract the largest distance, and assign masked inter-resiude as the largest distance in D
4. top pick k nearest neighbors
5. output the k nearest distance and according neighbour idx

Return:
D_neighbors: k nearest distances
E_idx: according neighbour idx


In [ ]:
def _dist(self, X, mask, eps=1E-6):
    mask_2D = torch.unsqueeze(mask,1) * torch.unsqueeze(mask,2)
    dX = torch.unsqueeze(X,1) - torch.unsqueeze(X,2)
    D = mask_2D * torch.sqrt(torch.sum(dX**2, 3) + eps)
    D_max, _ = torch.max(D, -1, keepdim=True)
    D_adjust = D + (1. - mask_2D) * D_max
    sampled_top_k = self.top_k
    D_neighbors, E_idx = torch.topk(D_adjust, np.minimum(self.top_k, X.shape[1]), dim=-1, largest=False)
    return D_neighbors, E_idx

In [18]:
from model_utils import featurize
import torch
import numpy as np

batch = [
    {
        'seq_chain_A': 'MKLVFLVLLVFVQGFG',
        'coords_chain_A': {'N_chain_A': np.random.rand(16, 3), 'CA_chain_A': np.random.rand(16, 3), 'C_chain_A': np.random.rand(16, 3), 'O_chain_A': np.random.rand(16, 3)},
        'seq_chain_B': 'MSVKVEEVG',
        'coords_chain_B': {'N_chain_B': np.random.rand(9, 3), 'CA_chain_B': np.random.rand(9, 3), 'C_chain_B': np.random.rand(9, 3), 'O_chain_B': np.random.rand(9, 3)},
        'seq_chain_C': 'ATCGATCGATCGATCG',
        'coords_chain_C': {'N_chain_C': np.random.rand(16, 3), 'CA_chain_C': np.random.rand(16, 3), 'C_chain_C': np.random.rand(16, 3), 'O_chain_C': np.random.rand(16, 3)},
        'masked_list': ['A', 'B'],
        'visible_list': ['C'],
        'num_of_chains': 3,
        'seq': 'MKLVFLVLLVFVQGFG'+ 'MSVKVEEVG' + 'ATCGATCGATCGATCG'
    },
      {
        'seq_chain_X': 'ACDEFGHIKLMNPQRSTVWY',
        'coords_chain_X': {'N_chain_X': np.random.rand(20, 3), 'CA_chain_X': np.random.rand(20, 3), 'C_chain_X': np.random.rand(20, 3), 'O_chain_X': np.random.rand(20, 3)},
        'seq_chain_Y': 'ADCDEFGHICKLM',
        'coords_chain_Y': {'N_chain_Y': np.random.rand(13, 3), 'CA_chain_Y': np.random.rand(13, 3), 'C_chain_Y': np.random.rand(13, 3), 'O_chain_Y': np.random.rand(13, 3)},
        'seq_chain_Z': 'CKLMNAPQRST',
        'coords_chain_Z': {'N_chain_Z': np.random.rand(11, 3), 'CA_chain_Z': np.random.rand(11, 3), 'C_chain_Z': np.random.rand(11, 3), 'O_chain_Z': np.random.rand(11, 3)},
        'masked_list': ['X', 'Y'],
        'visible_list': ['Z'],
        'num_of_chains': 3,
        'seq': 'ACDEFGHIKLMNPQRSTVWY'+'ADCDEFGHICKLM'+'CKLMNAPQRST'

    }
]

In [11]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all=featurize(batch, device
)

In [22]:
print(X.shape)
print(mask.shape)

torch.Size([2, 44, 4, 3])
torch.Size([2, 44])


In [23]:
mask

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.]])

In [26]:
mask_2D = torch.unsqueeze(mask,1) * torch.unsqueeze(mask,2)  # i, j
print(mask_2D.shape)
print(torch.unsqueeze(mask,1).shape)
print(torch.unsqueeze(mask,2).shape)

torch.Size([2, 44, 44])
torch.Size([2, 1, 44])
torch.Size([2, 44, 1])


In [28]:
dX = torch.unsqueeze(X,1) - torch.unsqueeze(X,2)  # distance between residues
print(dX.shape)
print(torch.unsqueeze(X,1).shape)
print(torch.unsqueeze(X,2).shape)

torch.Size([2, 44, 44, 4, 3])
torch.Size([2, 1, 44, 4, 3])
torch.Size([2, 44, 1, 4, 3])


In [51]:
eps=1E-6
CA_dX=dX[:,:,:,1,:]  # B,N,N,1, 3
D=torch.sqrt(torch.sum(CA_dX**2, 3) + eps) # sum over X, Y, Z
D.shape

torch.Size([2, 44, 44])

In [52]:
#mask out inter-residues that are padding or unvalid
D= mask_2D*D

In [53]:
D_max, _ = torch.max(D, -1, keepdim=True) # extract the largest distance
# adjust the distance info so that the masked inter residue distance is the largetst distance
D_adjust = D + (1. - mask_2D) * D_max

In [54]:
top_k=5  #5 nearest neighbors
D_neighbors, E_idx = torch.topk(D_adjust, np.minimum(top_k, X.shape[1]), dim=-1, largest=False)

In [55]:
D_neighbors.shape

torch.Size([2, 44, 5])

In [56]:
print(E_idx.shape)
print(E_idx[0,0,:]) # nearest neighbor index for batch 0, residue 0

torch.Size([2, 44, 5])
tensor([ 0, 18, 20,  1,  8])


#7.2 Second Function
RBF function

---
Input: D-Distance info  [B, N, K]
---
Function explain: calculate RBF kernel function\
generate #num_rbf values linearly between 2 and 22 Å\
calculate scaling factor D_sigma \
then calculate RBF value for all different RBF kernel functions (Gaussian) \

Overall, the distance info has been encoded into a #num_rbf dimension vector

---
Return: RBF value [B, N, K, num_rbf]

In [44]:
def _rbf(self, D):
    device = D.device
    D_min, D_max, D_count = 2., 22., self.num_rbf
    D_mu = torch.linspace(D_min, D_max, D_count, device=device)
    D_mu = D_mu.view([1,1,1,-1])
    D_sigma = (D_max - D_min) / D_count
    D_expand = torch.unsqueeze(D, -1)
    RBF = torch.exp(-((D_expand - D_mu) / D_sigma)**2)
    return RBF

In [61]:
num_rbf = 100
D_min, D_max, D_count = 2., 22., num_rbf
D_mu = torch.linspace(D_min, D_max, D_count, device=device)
D_mu = D_mu.view([1,1,1,-1])  # the mean value in Guassian RBF kenerl

print(D_mu)
print(D_mu.shape)

tensor([[[[ 2.0000,  2.2020,  2.4040,  2.6061,  2.8081,  3.0101,  3.2121,
            3.4141,  3.6162,  3.8182,  4.0202,  4.2222,  4.4242,  4.6263,
            4.8283,  5.0303,  5.2323,  5.4343,  5.6364,  5.8384,  6.0404,
            6.2424,  6.4444,  6.6465,  6.8485,  7.0505,  7.2525,  7.4545,
            7.6566,  7.8586,  8.0606,  8.2626,  8.4646,  8.6667,  8.8687,
            9.0707,  9.2727,  9.4747,  9.6768,  9.8788, 10.0808, 10.2828,
           10.4848, 10.6869, 10.8889, 11.0909, 11.2929, 11.4949, 11.6970,
           11.8990, 12.1010, 12.3030, 12.5051, 12.7071, 12.9091, 13.1111,
           13.3131, 13.5152, 13.7172, 13.9192, 14.1212, 14.3232, 14.5253,
           14.7273, 14.9293, 15.1313, 15.3333, 15.5354, 15.7374, 15.9394,
           16.1414, 16.3434, 16.5455, 16.7475, 16.9495, 17.1515, 17.3535,
           17.5556, 17.7576, 17.9596, 18.1616, 18.3636, 18.5657, 18.7677,
           18.9697, 19.1717, 19.3737, 19.5758, 19.7778, 19.9798, 20.1818,
           20.3838, 20.5859, 20.7879, 

In [59]:
print(D_neighbors.shape)

torch.Size([2, 44, 5])


In [62]:
D_sigma = (D_max - D_min) / D_count   # scaling
D_expand = torch.unsqueeze(D_neighbors, -1)  # B, N, k, 1
print(D_expand.shape)


torch.Size([2, 44, 5, 1])


In [63]:
RBF = torch.exp(-((D_expand - D_mu) / D_sigma)**2) #calculated RBF kernel value for various means

In [67]:
print(RBF.shape)
print(RBF[0,0,1,:10])

torch.Size([2, 44, 5, 100])
tensor([2.0127e-40, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])


#7.3 Third Function
get_rbf function
---

---
Input: D-Distance info  [B, N, K]\

Function explain:
calculate distance between A and B based on E_idx\
convert distance to  #num_rbf values vector by calling rbf function\

Overall: encoding distance btween A and B atoms

---
Return: RBF value [B, N, K, num_rbf]

In [69]:
def _get_rbf(self, A, B, E_idx):
    D_A_B = torch.sqrt(torch.sum((A[:,:,None,:] - B[:,None,:,:])**2,-1) + 1e-6) #[B, L, L]
    D_A_B_neighbors = gather_edges(D_A_B[:,:,:,None], E_idx)[:,:,:,0] #[B,L,K]
    RBF_A_B = self._rbf(D_A_B_neighbors)
    return RBF_A_B


In [74]:
Ca = X[:,:,1,:]
N = X[:,:,0,:]
C = X[:,:,2,:]
O = X[:,:,3,:]
print(Ca.shape)

torch.Size([2, 44, 3])


In [76]:

D_A_B = torch.sqrt(torch.sum((N[:,:,None,:] - C[:,None,:,:])**2,-1) + 1e-6) #[B, L, L]
# first broadcast then calculate inter atom distance
print(D_A_B.shape)

torch.Size([2, 44, 44])


In [78]:
# then gather distance from K-nearest neighbors
from model_utils import gather_edges
D_A_B_neighbors = gather_edges(D_A_B[:,:,:,None], E_idx)[:,:,:,0] #[B,L,K]

def _rbf(D):
    device = D.device
    D_min, D_max, D_count = 2., 22., 100
    D_mu = torch.linspace(D_min, D_max, D_count, device=device)
    D_mu = D_mu.view([1,1,1,-1])
    D_sigma = (D_max - D_min) / D_count
    D_expand = torch.unsqueeze(D, -1)
    RBF = torch.exp(-((D_expand - D_mu) / D_sigma)**2)
    return RBF

#then encode the distance through RBF values
RBF_A_B =_rbf(D_A_B_neighbors)

In [81]:
print(RBF_A_B.shape) # B, N, k, num_of_rbf
print(RBF_A_B[0,0,1,:10])

torch.Size([2, 44, 5, 100])
tensor([2.4677e-17, 3.3394e-23, 5.8723e-30, 1.3419e-37, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])


#Rest of ProteinFeature
Combine Postional embedding and geometry embedding


In [83]:
offset = residue_idx[:,:,None]-residue_idx[:,None,:]  # [B,N,N] calculate interresidue idx distance
offset = gather_edges(offset[:,:,:,None], E_idx)[:,:,:,0] #[B, N, K] # extract only the K nearest
print(offset.shape)
print(offset[0,0,:])

torch.Size([2, 44, 5])
tensor([   0, -118, -120,   -1,   -8])


In [85]:
chain_labels=chain_encoding_all
print(chain_labels.shape)
d_chains = ((chain_labels[:, :, None] - chain_labels[:,None,:])==0).long() #find self vs non-self interaction
E_chains = gather_edges(d_chains[:,:,:,None], E_idx)[:,:,:,0] # [B, N, K]
#E_positional = self.embeddings(offset.long(), E_chains)
print(E_chains.shape)
print(E_chains[0,0,:]) # 1 is self interaction(within the same chain), 0 is inter-chain
#the results agree with the K-nearest neighbor indices [   0, -118, -120,   -1,   -8]

torch.Size([2, 44])
torch.Size([2, 44, 5])
tensor([1, 0, 0, 1, 1])


In [91]:
import torch.nn as nn
class PositionalEncodings(nn.Module):
    def __init__(self, num_embeddings, max_relative_feature=32):
        super(PositionalEncodings, self).__init__()
        self.num_embeddings = num_embeddings
        self.max_relative_feature = max_relative_feature
        self.linear = nn.Linear(2*max_relative_feature+1+1, num_embeddings)

    def forward(self, offset, mask):
        d = torch.clip(offset + self.max_relative_feature, 0, 2*self.max_relative_feature)*mask + (1-mask)*(2*self.max_relative_feature+1)
        d_onehot = torch.nn.functional.one_hot(d, 2*self.max_relative_feature+1+1)
        E = self.linear(d_onehot.float())
        return E
num_positional_embeddings=10
embeddings = PositionalEncodings(num_positional_embeddings)
E_positional = embeddings(offset.long(), E_chains) # E_chains act as a mask, inter chain is set to be 2*self.max_relative_feature+1+1
#E = torch.cat((E_positional, RBF_all), -1)
print(E_positional.shape) #[B,N,K,10]  trainable positional embedding
print(E_positional[0,0,1,:])

torch.Size([2, 44, 5, 10])
tensor([-0.0061, -0.1261, -0.0489,  0.0102,  0.0175,  0.0838,  0.0170,  0.1484,
         0.1981, -0.1855], grad_fn=<SliceBackward0>)


In [ ]:
# Finally Combind postional and geometry embedding
'''
RBF_all = []
RBF_all.append(self._rbf(D_neighbors)) #Ca-Ca
RBF_all.append(self._get_rbf(N, N, E_idx)) #N-N
RBF_all.append(self._get_rbf(C, C, E_idx)) #C-C
RBF_all.append(self._get_rbf(O, O, E_idx)) #O-O
RBF_all.append(self._get_rbf(Cb, Cb, E_idx)) #Cb-Cb
RBF_all.append(self._get_rbf(Ca, N, E_idx)) #Ca-N
RBF_all.append(self._get_rbf(Ca, C, E_idx)) #Ca-C
RBF_all.append(self._get_rbf(Ca, O, E_idx)) #Ca-O
RBF_all.append(self._get_rbf(Ca, Cb, E_idx)) #Ca-Cb
RBF_all.append(self._get_rbf(N, C, E_idx)) #N-C
RBF_all.append(self._get_rbf(N, O, E_idx)) #N-O
RBF_all.append(self._get_rbf(N, Cb, E_idx)) #N-Cb
RBF_all.append(self._get_rbf(Cb, C, E_idx)) #Cb-C
RBF_all.append(self._get_rbf(Cb, O, E_idx)) #Cb-O
RBF_all.append(self._get_rbf(O, C, E_idx)) #O-C
RBF_all.append(self._get_rbf(N, Ca, E_idx)) #N-Ca
RBF_all.append(self._get_rbf(C, Ca, E_idx)) #C-Ca
RBF_all.append(self._get_rbf(O, Ca, E_idx)) #O-Ca
RBF_all.append(self._get_rbf(Cb, Ca, E_idx)) #Cb-Ca
RBF_all.append(self._get_rbf(C, N, E_idx)) #C-N
RBF_all.append(self._get_rbf(O, N, E_idx)) #O-N
RBF_all.append(self._get_rbf(Cb, N, E_idx)) #Cb-N
RBF_all.append(self._get_rbf(C, Cb, E_idx)) #C-Cb
RBF_all.append(self._get_rbf(O, Cb, E_idx)) #O-Cb
RBF_all.append(self._get_rbf(C, O, E_idx)) #C-O
RBF_all = torch.cat(tuple(RBF_all), dim=-1)
E = torch.cat((E_positional, RBF_all), -1) # dimension: 25 (num of inter atom pairs) * num_rbf + num_positional_embeddings
edge_in = 25*num_rbf + num_positional_embeddings
self.edge_embedding = nn.Linear(edge_in, edge_features, bias=False) # MLP to edge_feature
self.norm_edges = nn.LayerNorm(edge_features) # normalization

Return E    # Edge feature embedding

'''


